## Import necessary packages

In [4]:
import numpy as np
import pandas as pd
import math
import pywt

## Thermal Comfort Calculation

Thermal comfort is a product of Temperature and Humidity. In order to determine Thermal Comfort, the Heat Index is calculated.

Calculation is based on research paper [1], which provides with a formula allowing to calculate Heat Index, that is an apparent temperature (temperature that is perceived) as a function of Humidity and Temperature. It considers previous research and estimations and compares to existing models used by Weather Services (NWS), that use least squares in order to fit the data using a polynomial function and two variables. Author obtains very good results and manages to extrapolate the equation to lower temperatures accurately, whereas in previous research HI Index would reasonable only above 21 degrees Celsius. Squared Error was reduced significantly with maximum error of 2 degree Celsius at 48 degrees.

For the purpose of this experiment interest is put on temperatures in a range 15-35, for which good results were obtained.

The formula for calculating the Apparent Tmeperature, known as the Heat Index is calculated using a formula derived below:

$$HeatIndex = Temperature - 1.0799 \cdot \exp^{0.03755 \cdot Temperature} \cdot (1 - \exp^{0.0801 \cdot (DewPoint - 14)})$$

Equation takes into account Dewpoint, which is calculated using a Magnus–Tetens formula, that converts the Temperature (Celcius) and Humidity to Dewpoint:

$$DewPoint = \frac{b \cdot \alpha}{a - \alpha}$$ where,

$$\alpha = \frac{a \cdot Temperature}{b + Temperature} + \ln(Humidity)$$

where, $a$ and $b$ are constant equal to $17.27$ and $237.3$, respectively.

[1] Schoen, C. (2005). A new empirical model of the temperature–humidity index. Journal of applied meteorology, 44(9), 1413-1420.

In [3]:
def HI_calculation(sensor_temperature, sensor_humidity, number_of_samples):
    """
    :param sensor_temperature: array of temperature readings
    :param sensor_humidity: array of humidity readings
    :param number_of_samples: number of generated measurements
    :return: Heat Index
    """
    heat_index = []
    a: float = 17.27
    b: float = 237.3

    for cell in range(number_of_samples):
        alfa = ((a * sensor_temperature[cell]) / (b + sensor_temperature[cell])) + math.log(
            sensor_humidity[cell] / 100)

        D = (b * alfa) / (a - alfa)

        HI = sensor_temperature[cell] - 1.0799 * math.exp(0.03755 * sensor_temperature[cell]) * (
                1 - math.exp(0.0801 * (D - 14)))

        heat_index.append(HI)

    return heat_index

## Generating the User Satisfaction Scores

In order to resemble the real-life scores company's KPIs were used to generate the Satisfaction Score.
These KPI are based on research and WELL Building Standard, that evaluate users comfort and performance over various parameters.

"The WELL Building Standard® is a performance-based system for measuring, certifying, and monitoring features of the built environment that impact human health and wellbeing, through air, water, nourishment, light, fitness, comfort, and mind" [1]. In other words WELL Building Standard defines, among other things the indoor conditions that create healthy and productive conditions for the tenats.

Synthetic Scores are generated by taking a random number in a given interval that corresponds to the WELL Standard for a particular sensor reading.

Function takes the generated sensor measurement as and input and based on them assignes the appropriate score.

Complete methodology of dentermining the appropriate Satisfaction Scores is listed below.


### Apparent Temperature - Thermal Comfort (WELL based)

#### Calculations Reference Scale
<table
  cellpadding="0"
  cellspacing="0"
  style="table-layout: fixed; text-align: center; width: 700px; font-size: 11px; border: 0;"
>
  <tr style="font-weight: bold;">
    <td colspan="1" style="border: 0;">&nbsp;</td>
    <td colspan="2" style="border: 0;">17</td>
    <td colspan="2" style="border: 0;">19</td>
    <td colspan="2" style="border: 0;">21</td>
    <td colspan="2" style="border: 0;">22</td>
    <td colspan="1" style="border: 0; font-size: 13px;"></td>
    <td colspan="2" style="border: 0;">24</td>
    <td colspan="2" style="border: 0;">25</td>
    <td colspan="2" style="border: 0;">27</td>
    <td colspan="2" style="border: 0;">29</td>
    <td colspan="1" style="border: 0;">&nbsp;</td>
  </tr>
  <tr>
    <td colspan="2" style="background-color: red; border: 0;">0</td>
    <td colspan="2" style="background-color: darkorange; border: 0;">0.25</td>
    <td colspan="2" style="background-color: gold; border: 0;">0.5</td>
    <td colspan="2" style="background-color: greenyellow; border: 0;">0.75</td>
    <td colspan="3" style="background-color: green; border: 0;">1</td>
    <td colspan="2" style="background-color: greenyellow; border: 0;">0.75</td>
    <td colspan="2" style="background-color: gold; border: 0;">0.5</td>
    <td colspan="2" style="background-color: darkorange; border: 0;">0.25</td>
    <td colspan="2" style="background-color: red; border: 0;">0</td>
  </tr>
</table>

#### Calculations Reference Table
| Rule | KPI Value |
|--|--|
| [value] > 22 && [value] <= 24 | 1 |
| [value] > 21 && [value] <= 22 | 0.75 |
| [value] > 24 && [value] <= 25 | 0.75 |
| [value] > 17 && [value] <= 21 | 0.5 |
| [value] > 25 && [value] <= 27 | 0.5 |
| [value] > 19 && [value] <= 17 | 0.25 |
| [value] > 27 && [value] <= 29 | 0.25 |
| [value] < 19 && [value] >= 0 | 0 |
| [value] > 29 && [value] < 40 | 0 |
| [value] < 0 | Sensor Malfunction |
| [value] => 40 | Sensor Malfunction |
| [value] = BLANK | null |

---

### CO2 (WELL based)

#### Calculations Reference Scale
<table
  cellpadding="0"
  cellspacing="0"
  style="table-layout: fixed; text-align: center; width: 700px; font-size: 11px; border: 0;"
>
  <tr style="font-weight: bold;">
    <td colspan="2" style="border: 0;">0</td>
    <td colspan="2" style="border: 0;">650</td>
    <td colspan="2" style="border: 0;">800</td>
    <td colspan="2" style="border: 0;">1000</td>
    <td colspan="2" style="border: 0;">1200</td>
    <td colspan="2" style="border: 0;">1500</td>
    <td colspan="2" style="border: 0;">1800</td>
    <td colspan="2" style="border: 0;">2000</td>
    <td colspan="1" style="border: 0;">&nbsp;</td>
  </tr>
  <tr>
    <td colspan="0" style="background-color: none; border: 0;"></td>
    <td colspan="2" style="background-color: green; border: 0;">1</td>
    <td colspan="2" style="background-color: lightgreen; border: 0;">0.9</td>
    <td colspan="2" style="background-color: greenyellow; border: 0;">0.75</td>
    <td colspan="2" style="background-color: gold; border: 0;">0.5</td>
    <td colspan="2" style="background-color: darkorange; border: 0;">0.25</td>
    <td colspan="2" style="background-color: red; border: 0;">0.15</td>
    <td colspan="2" style="background-color: darkred; border: 0;">0</td>
  </tr>
</table>


#### Calculations Reference Table
| Rule | KPI Value |
|--|--|
| [value] <= 650 | 1 |
| [value] > 650 && [value] <= 800 | 0.9 |
| [value] > 800 && [value] <= 1000 | 0.75 |
| [value] > 1000 && [value] <= 1200 | 0.5 |
| [value] > 1200 && [value] <= 1500 | 0.25 |
| [value] > 1500 && [value] <= 1800 | 0.15 |
| [value] > 1800 | 0.0 |
| [value] < 0 | sensor malfunction |
| [value] > 2500 | sensor malfunction |
| [value] = BLANK | null |

---

### Light (WELL based)


#### Calculations Reference Scale
<table
  cellpadding="0"
  cellspacing="0"
  style="table-layout: fixed; text-align: center; width: 700px; font-size: 11px; border: 0;"
>
  <tr style="font-weight: bold;">
    <td colspan="0" style="border: 0;"></td>
    <td colspan="2" style="border: 0;">0</td>
    <td colspan="2" style="border: 0;">300</td>
    <td colspan="2" style="border: 0;">450</td>
    <td colspan="2" style="border: 0;">600</td>
    <td colspan="1" style="border: 0;">1000</td>
  </tr>
  <tr>
    <td colspan="0" style="background-color: none; border: 0;"></td>
    <td colspan="2" style="background-color: red; border: 0;">0</td>
    <td colspan="2" style="background-color: darkorange; border: 0;">0.25</td>
    <td colspan="2" style="background-color: gold; border: 0;">0.5</td>
    <td colspan="2" style="background-color: yellowgreen; border: 0;">0.75</td>
    <td colspan="2" style="background-color: green; border: 0;">1</td>
  </tr>
</table>

#### Calculations Reference Table
| Rule | KPI Value |
|--|--|
| [value] >= 1000 | 1 |
| [value] >= 600 && [value] < 1000 | 0.75 |
| [value] >= 450 && [value] < 600 | 0.5 |
| [value] >= 300 && [value] < 450 | 0.25 |
| [value] >= 0 && [value] < 300 | 0.0 |
| [value] < 0 | sensor malfunction |
| [value] > 25000 | sensor malfunction |
| [value] = BLANK | null |

---

### Noise
The Noise (dB) KPI is based on "Kentallen binnenmilieu & productiviteit ten behoeve van de EET value case tool" by BBA 

#### Calculations Reference Scale
<table
  cellpadding="0"
  cellspacing="0"
  style="table-layout: fixed; text-align: center; width: 700px; font-size: 11px; border: 0;"
>
  <tr style="font-weight: bold;">
    <td colspan="0" style="border: 0;"></td>
    <td colspan="2" style="border: 0;">0</td>
    <td colspan="2" style="border: 0;">55</td>
    <td colspan="2" style="border: 0;">60</td>
    <td colspan="2" style="border: 0;">65</td>
    <td colspan="1" style="border: 0;">70</td>
  </tr>
  <tr>
    <td colspan="0" style="background-color: none; border: 0;"></td>
    <td colspan="2" style="background-color: green; border: 0;">1</td>
    <td colspan="2" style="background-color: yellowgreen; border: 0;">0.75</td>
    <td colspan="2" style="background-color: gold; border: 0;">0.5</td>
    <td colspan="2" style="background-color: darkorange; border: 0;">0.25</td>
    <td colspan="2" style="background-color: red; border: 0;">0</td>
  </tr>
</table>

#### Calculations Reference Table
| Rule | KPI Value |
|--|--|
| [value] <= 55 | 1 |
| [value] > 55 && [value] <= 60 | 0.75 |
| [value] > 60 && [value] <= 65 | 0.5 |
| [value] > 65 && [value] <= 70 | 0.25 |
| [value] > 70 | 0.0 |
| [value] < 0 | sensor malfunction |
| [value] > 150 | sensor malfunction |
| [value] = BLANK | null |


[1] https://v2.wellcertified.com/v/en/air/feature/12

In [4]:
def generate_satisfaction_scores(generated_data, number_of_samples):
    """
    :param generated_data: array of generated sensor measurements
    :param number_of_samples: number of generated measurements
    :return: array with sensors and generated satisfaction scores
    """

    for cell in range(number_of_samples):

        ### Temperature
        if (generated_data[cell, 0] > 17) and (generated_data[cell, 0] <= 29):
            generated_data[cell, 6] = np.random.uniform(0.2, 0.4)

        if (generated_data[cell, 0] > 19) and (generated_data[cell, 0] <= 27):
            generated_data[cell, 6] = np.random.uniform(0.4, 0.6)

        if (generated_data[cell, 0] > 21) and (generated_data[cell, 0] <= 25):
            generated_data[cell, 6] = np.random.uniform(0.6, 0.8)

        if (generated_data[cell, 0] > 22) and (generated_data[cell, 0] <= 24):
            generated_data[cell, 6] = np.random.uniform(0.8, 1)

        ### Humidity
        if (generated_data[cell, 1] > 15) and (generated_data[cell, 1] <= 85):
            generated_data[cell, 7] = np.random.uniform(0.2, 0.4)

        if (generated_data[cell, 1] > 20) and (generated_data[cell, 1] <= 80):
            generated_data[cell, 7] = np.random.uniform(0.4, 0.6)

        if (generated_data[cell, 1] > 25) and (generated_data[cell, 1] <= 75):
            generated_data[cell, 7] = np.random.uniform(0.6, 0.8)

        if (generated_data[cell, 1] > 30) and (generated_data[cell, 1] <= 70):
            generated_data[cell, 7] = np.random.uniform(0.8, 1)

        ### Apparent temperature - Heat Index (Temperature accounted for humidity)
        if (generated_data[cell, 2] > 17) and (generated_data[cell, 2] <= 29):
            generated_data[cell, 8] = np.random.uniform(0.2, 0.4)

        if (generated_data[cell, 2] > 19) and (generated_data[cell, 2] <= 27):
            generated_data[cell, 8] = np.random.uniform(0.4, 0.6)

        if (generated_data[cell, 2] > 21) and (generated_data[cell, 2] <= 25):
            generated_data[cell, 8] = np.random.uniform(0.6, 0.8)

        if (generated_data[cell, 2] > 22) and (generated_data[cell, 2] <= 24):
            generated_data[cell, 8] = np.random.uniform(0.8, 1)

        ### CO2
        if generated_data[cell, 3] <= 1800:
            generated_data[cell, 9] = np.random.uniform(0.1, 0.2)

        if (generated_data[cell, 3] > 1200) and (generated_data[cell, 3] <= 1500):
            generated_data[cell, 9] = np.random.uniform(0.2, 0.4)

        if (generated_data[cell, 3] > 1000) and (generated_data[cell, 3] <= 1200):
            generated_data[cell, 9] = np.random.uniform(0.4, 0.6)

        if (generated_data[cell, 3] > 800) and (generated_data[cell, 3] <= 1000):
            generated_data[cell, 9] = np.random.uniform(0.6, 0.8)

        if (generated_data[cell, 3] > 650) and (generated_data[cell, 3] <= 800):
            generated_data[cell, 9] = np.random.uniform(0.8, 0.9)

        if generated_data[cell, 3] <= 650:
            generated_data[cell, 9] = np.random.uniform(0.9, 1)

        ### Light
        if generated_data[cell, 4] >= 300:
            generated_data[cell, 10] = np.random.uniform(0.2, 0.4)

        if generated_data[cell, 4] >= 450:
            generated_data[cell, 10] = np.random.uniform(0.4, 0.6)

        if generated_data[cell, 4] >= 600:
            generated_data[cell, 10] = np.random.uniform(0.6, 0.8)

        if generated_data[cell, 4] >= 1000:
            generated_data[cell, 10] = np.random.uniform(0.8, 1)

        ### Noise
        if generated_data[cell, 5] <= 70:
            generated_data[cell, 11] = np.random.uniform(0.2, 0.4)

        if generated_data[cell, 5] <= 65:
            generated_data[cell, 11] = np.random.uniform(0.4, 0.6)

        if generated_data[cell, 5] <= 60:
            generated_data[cell, 11] = np.random.uniform(0.6, 0.8)

        if generated_data[cell, 5] <= 55:
            generated_data[cell, 11] = np.random.uniform(0.8, 1)

    return generated_data

## NULL Model

This model is used for statistical testing and power simulation. This model doesnt impose any constaints. It generates a random number form a uniform distribution between 0 and 100% to assign a random score to each sensor reading.

In [5]:
def generate_null_model(generated_data, number_of_samples):
    """
    :param generated_data: array of generated sensor measurements
    :param number_of_samples: number of generated measurements
    :return: array with sensors and random satisfaction scores
    """

    for cell in range(number_of_samples):

    ### Temperature
        generated_data[cell, 6] = np.random.uniform(0, 1)
    ### Humidity    
        generated_data[cell, 7] = np.random.uniform(0, 1)
    ### Heat Index
        generated_data[cell, 8] = np.random.uniform(0, 1)
    ### CO2
        generated_data[cell, 9] = np.random.uniform(0, 1)
    ### Light
        generated_data[cell, 10] = np.random.uniform(0, 1)
    ### Noise        
        generated_data[cell, 11] = np.random.uniform(0, 1)

    return generated_data

## Smooth Generated Satisfaction Scores

Scores generated by applying funstion **generate_satisfaction_scores** are a step function which is not smooth and doesn't reperesent the data realistically. 
Function **smoothing** will take the column with scores and apply Wavelet Transformation in order to smooth the data and receive a trend. 

Belowe there is an example on how the smoothing looks like for db8 and noise standard deviation od 0.02.

Plot displaying CO2 vs Satisfaction Score **before** smoothing

<img src="co2_not_smoothed.png" height="600" width="600">


Plot displaying CO2 vs Satisfaction Score **after** smoothing

<img src="co2_smoothed.png" height="600" width="600">


In [6]:
def smoothing(y_values: np.ndarray, number_of_samples, wavelet):
    """
    Method that smooths a time series using wavelet transforms.

    ```
    :param y_values: y values of the original time series
    :param number_of_samples: desired length of the results
    :param wavelet: type of wavelet i.e. db2, db8 etc.
    :return: smoothed data series
    """

    levels = pywt.dwt_max_level(y_values.shape[0], wavelet)

    # Decompose getting only the details
    for _ in range(levels):
        y_values = pywt.downcoef(part='a', data=y_values, wavelet=wavelet)

    for _ in range(levels):
        details = np.zeros(y_values.shape)
        y_values = pywt.idwt(y_values, details, wavelet=wavelet)

    return y_values[:number_of_samples]

# Main function - Data Set Generation

This function generates sensor measurements taking a random number from an uniform dirstribution given the interval of possible values. Values that are defined can be registered in reality and ***not*** as a result of a sensor failure. Heat Index is calculated by calling the function **HI_calculation**.

The probability density function for the uniform distribution is:

$$p(x) = \frac{1}{b - a}$$  

for $x \in [a, b]$ with zero elsewhere.

Once possible sensor measurements are generated, function **generate_satisfaction_scores** is called to assign the Satisfaction Scores.
In order to make data more realistic, function is smoothed and gaussian noise is added to the generated data with a given standard deviation.
Function returs a complete generated synthetic dataset based on the constraints imposed on the sensor readigns by the WELL Standard.

In [7]:
def generate_dataset_with_sensor_readings_and_satisfaction_scores(number_of_samples, snr, wavelet):
    """
    :param number_of_samples: number of generated measurements
    :param noise_standard_deviation: standard deviation of a noise added to the generated sample
    :param wavelet: type of wavelet
    :return: generated dataset
    """
    # Generate a random number from an uniform distribution in a range given by KPIs

    sensor_temperature = [np.random.uniform(15, 32) for _ in range(number_of_samples)]
    sensor_humidity = [np.random.uniform(20, 85) for _ in range(number_of_samples)]
    sensor_heat_index = HI_calculation(sensor_temperature, sensor_humidity, number_of_samples)
    sensor_co2 = [np.random.uniform(300, 2300) for _ in range(number_of_samples)]
    sensor_light = [np.random.uniform(50, 1500) for _ in range(number_of_samples)]
    sensor_noise = [np.random.uniform(30, 75) for _ in range(number_of_samples)]

    # Create an ndarray with all generated sensor measurements and fill the new columns of ndarray with the random
    # values between 15% and 20% representing guaranteed minimal satisfaction in each category

    generated_data = np.asarray(
        list(zip(sensor_temperature, sensor_humidity, sensor_heat_index, sensor_co2, sensor_light, sensor_noise,
                 [np.random.uniform(0.1, 0.2) for _ in range(number_of_samples)],
                 [np.random.uniform(0.1, 0.2) for _ in range(number_of_samples)],
                 [np.random.uniform(0.1, 0.2) for _ in range(number_of_samples)],
                 [np.random.uniform(0.1, 0.2) for _ in range(number_of_samples)],
                 [np.random.uniform(0.1, 0.2) for _ in range(number_of_samples)],
                 [np.random.uniform(0.1, 0.2) for _ in range(number_of_samples)])), dtype=np.float32)

    generated_data_with_scores = generate_satisfaction_scores(generated_data, number_of_samples)

    for i in range(6, 12):
        generated_data_with_scores = generated_data_with_scores[generated_data_with_scores[:, i - 6].argsort()]
        generated_data_with_scores[:, i] = smoothing(generated_data_with_scores[:, i], number_of_samples, wavelet)

    # mean = 0
    # noise = np.random.normal(mean, noise_standard_deviation, [number_of_samples, 6])

    def generate_noise(snr):
        if snr != 0.0:
            noise = np.random.normal(size=number_of_samples)
            # work out the current SNR
            current_snr = np.mean(generated_data_with_scores[:, 6:]) / np.std(noise)
            # scale the noise by the snr ratios (smaller noise <=> larger snr)
            noise *= (current_snr / snr)
        else:
            noise = np.zeros(number_of_samples)
        # return the new signal with noise
        return noise

    # signal_to_noise_ratio = np.mean(generated_data_with_scores[:, 6:]) / noise_standard_deviation

    signal_to_noise_ratio = generate_noise(snr)

    for i in range(6, 12):
        generated_data_with_scores[:, i] = generated_data_with_scores[:, i] - np.abs(signal_to_noise_ratio)

    # outisde bounds
    generated_data_with_scores[:, 6:][np.where(generated_data_with_scores[:, 6:] < 0)] = np.random.uniform(0, 0.05)
    generated_data_with_scores[:, 6:][np.where(generated_data_with_scores[:, 6:] < 0)] = np.random.uniform(0, 0.05)

    satisfaction_vs_sensors = pd.DataFrame(generated_data_with_scores,
                                           columns=['sensor_temperature', 'sensor_humidity', 'sensor_heat_index',
                                                    'sensor_air', 'sensor_light', 'sensor_noise',
                                                    'comfort_score_temperature', 'comfort_score_humidity',
                                                    'comfort_score_heat_index', 'comfort_score_air',
                                                    'comfort_score_light', 'comfort_score_noise'])

    generated_data_null = generate_null_model(generated_data, number_of_samples)

    satisfaction_vs_sensors_null = pd.DataFrame(generated_data_null,
                                                columns=['sensor_temperature', 'sensor_humidity', 'sensor_heat_index',
                                                         'sensor_air', 'sensor_light', 'sensor_noise',
                                                         'comfort_score_temperature', 'comfort_score_humidity',
                                                         'comfort_score_heat_index', 'comfort_score_air',
                                                         'comfort_score_light', 'comfort_score_noise'])
    
    return satisfaction_vs_sensors, satisfaction_vs_sensors_null

## Generating data - Example

In [8]:
number_of_samples = 1000
signal_to_noise_ratio = 3.5
wavelet = 'db8'

satisfaction_vs_sensors, satisfaction_vs_sensors_null = generate_dataset_with_sensor_readings_and_satisfaction_scores(
    number_of_samples, signal_to_noise_ratio, wavelet)

,sensor_temperature,sensor_humidity,sensor_heat_index,sensor_air,sensor_light,sensor_noise,comfort_score_temperature,comfort_score_humidity,comfort_score_heat_index,comfort_score_air,comfort_score_light,comfort_score_noise
0,18.451464,54.442871,17.750771,357.025330,589.208984,36.236168,0.357933,0.486382,0.761150,0.235031,0.837259,0.870551
1,20.608095,48.619415,19.887993,1029.991089,468.799805,40.798237,0.641439,0.714118,0.043745,0.844676,0.671256,0.214239
2,26.755348,80.421265,29.919067,256.932037,1493.497559,68.829697,0.701538,0.391821,0.256218,0.133922,0.338449,0.901746
3,28.927155,50.777706,30.031160,1641.508911,554.558899,63.108376,0.363107,0.411203,0.319647,0.661028,0.904055,0.849373
4,27.230074,67.354073,29.341959,893.853699,1101.222656,41.255516,0.976973,0.906786,0.506873,0.336229,0.502193,0.086300
